In [34]:
import numpy as np 
import pandas as pd 

import nltk
import json

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering


In [16]:
target = pd.read_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/target.csv')
print(len(target))
target.head()

8703


,id,date,2,3,5,6,7,8,9,10,...,19,34,35,37,41,46,P1,P4,P12,P7
0,HUDOC-ECHR-1999-001-58225,1999-03-25,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
1,HUDOC-ECHR-1999-001-58226,1999-03-25,-1,-1,-1,0,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
2,HUDOC-ECHR-1999-001-58227,1999-03-25,-1,-1,-1,2,-1,2,-1,-1,...,-1,-1,2,-1,2,-1,2,-1,-1,-1
3,HUDOC-ECHR-1999-001-58239,1999-04-29,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1
4,HUDOC-ECHR-1999-001-58251,1999-05-20,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1


In [19]:
stopwords = 'english'
part = 'full'
article = '6'
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}/article{}_{}.csv"

ngrams = pd.read_csv(path.format(stopwords,article,part))
ngrams.head()

,able,abroad,absence,abuse,acar,accept,accepted,access,access court,accident,...,written,wrote,year,years,years imprisonment,young,zagreb,от,по,target
0,0,0,0,1,0,0,0,0,0,0,...,1,0,1,1,1,0,0,0,0,nonviolation
1,0,0,0,1,0,1,2,1,0,0,...,4,0,2,13,4,2,0,0,0,nonviolation
2,0,0,0,0,0,1,0,1,0,0,...,5,0,1,6,5,0,0,0,0,nonviolation
3,1,0,1,1,0,0,0,1,0,0,...,5,1,0,19,7,0,0,0,0,nonviolation
4,1,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,nonviolation


In [79]:
ngrams = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/echr_dataset/Article3/ngrams_a3_full.csv",header=None)
print(len(ngrams))
ngrams.head()

250


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.000000,0.000000,0.0,0.0,0.0,0.000367,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.001836,0.001101,0.0,0.000000,0.000367,0.000367,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.003125,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000562,0.000000,0.000000,0.0,0.000000,0.000562,0.000000,0.0,0.0
3,0.002523,0.001009,0.0,0.0,0.0,0.002018,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000505,0.000000,0.0,0.001009,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.001536,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [80]:
df = ngrams

In [71]:
#df = ngrams.drop(labels=['target'],axis=1)
#df = (df.T / df.T.sum()).T

In [83]:
#Transpose 
ngram_vectors = df.transpose().values

In [85]:
#Get similarity matrix
similarity = cosine_similarity(ngram_vectors)
similarity

array([[1.        , 0.62211031, 0.        , ..., 0.00321513, 0.037577  ,
        0.0076967 ],
       [0.62211031, 1.        , 0.        , ..., 0.00925753, 0.01316604,
        0.00273987],
       [0.        , 0.        , 1.        , ..., 0.07744365, 0.06270299,
        0.        ],
       ...,
       [0.00321513, 0.00925753, 0.07744365, ..., 1.        , 0.06004474,
        0.01765935],
       [0.037577  , 0.01316604, 0.06270299, ..., 0.06004474, 1.        ,
        0.        ],
       [0.0076967 , 0.00273987, 0.        , ..., 0.01765935, 0.        ,
        1.        ]])

In [100]:
#Get clusters 
clustering = SpectralClustering(30,n_neighbors=40,assign_labels="discretize").fit(similarity)
labels = clustering.labels_

In [101]:
pd.Series(labels).value_counts()

1     1161
2      175
3      155
20      82
9       74
28      49
21      37
5       35
11      29
26      27
4       22
0       15
22      14
6       12
8       12
16      12
14      11
27      10
7        9
25       9
23       8
10       6
18       6
13       5
17       5
19       5
24       5
29       4
12       3
15       3
dtype: int64

In [99]:
pd.Series(labels).value_counts()

0     1166
2      175
24     155
4       83
17      74
7       52
23      37
5       34
3       29
28      27
20      22
16      15
22      14
6       12
14      12
15      12
19      11
21      10
8        9
26       9
13       6
11       6
10       5
9        5
25       5
1        5
29       4
12       3
18       3
dtype: int64